In [ ]:
!pip install torch

In [33]:
import torch

inputs = torch.tensor([
    [0.43, 0.15, 0.891],  # "Your"
    [0.55, 0.87, 0.66],   # "journey"
    [0.57, 0.85, 0.641],  # "starts"
    [0.22, 0.58, 0.331], # "with"
    [0.77, 0.25, 0.101],  # "one"
    [0.05, 0.80, 0.551]   # "step"
])

print(inputs.shape)


torch.Size([6, 3])


In [41]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [44]:
torch.manual_seed(123)

W_Q = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_K = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_V = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)

In [49]:
query_2 = x_2 @ W_Q
value_2 = x_2 @ W_V
key_2 = x_2 @ W_K
key_2

tensor([0.4433, 1.1419])

In [52]:
queries = inputs @ W_Q
values = inputs @ W_V
keys = inputs @ W_K
keys

tensor([[0.3672, 0.7652],
        [0.4433, 1.1419],
        [0.4364, 1.1163],
        [0.2411, 0.6713],
        [0.1830, 0.3299],
        [0.3278, 0.9649]])

In [53]:
#calcate attention scores by dot producting query and key

attentionscore_2 = query_2 @ keys.T

attentionscore_2

tensor([1.2716, 1.8524, 1.8122, 1.0807, 0.5589, 1.5451])

In [54]:
attention_scores = queries @ keys.T
attention_scores

tensor([[0.9247, 1.3556, 1.3259, 0.7925, 0.4044, 1.1346],
        [1.2716, 1.8524, 1.8122, 1.0807, 0.5589, 1.5451],
        [1.2562, 1.8294, 1.7898, 1.0672, 0.5522, 1.5257],
        [0.6986, 1.0177, 0.9957, 0.5938, 0.3070, 0.8490],
        [0.6127, 0.8830, 0.8641, 0.5133, 0.2715, 0.7321],
        [0.9010, 1.3176, 1.2889, 0.7696, 0.3948, 1.1013]])

In [61]:
d_k = keys.shape[-1]

attention_weights = torch.softmax(attention_scores/d_k**0.5,dim=-1)
attention_weights

tensor([[0.1551, 0.2104, 0.2060, 0.1413, 0.1074, 0.1799],
        [0.1500, 0.2262, 0.2199, 0.1311, 0.0906, 0.1821],
        [0.1504, 0.2255, 0.2193, 0.1315, 0.0914, 0.1819],
        [0.1591, 0.1994, 0.1963, 0.1477, 0.1206, 0.1769],
        [0.1610, 0.1949, 0.1923, 0.1501, 0.1265, 0.1752],
        [0.1557, 0.2091, 0.2049, 0.1419, 0.1089, 0.1794]])

In [63]:
#context vector of x_2

x_2_context = attention_weights[1] @ values
x_2_context

tensor([0.3062, 0.8216])

In [67]:
#context vectors for all tokens

context_vectors =  attention_weights @ values
context_vectors

tensor([[0.2997, 0.8060],
        [0.3062, 0.8216],
        [0.3059, 0.8209],
        [0.2949, 0.7945],
        [0.2928, 0.7898],
        [0.2991, 0.8047]])

In [78]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
  def __init__(self,d_in,d_out):
    super().__init__()
    self.W_Q = nn.Parameter(torch.rand(d_in,d_out),requires_grad=True)
    self.W_V = nn.Parameter(torch.rand(d_in,d_out),requires_grad=True)
    self.W_K = nn.Parameter(torch.rand(d_in,d_out),requires_grad=True)
  def forward(self,x):
    keys = x @ self.W_K
    values = x @ self.W_V
    queries = x @ self.W_Q
    attention_scores = queries @ keys.T
    attention_weights = torch.softmax(attention_scores/keys.shape[-1]**0.5,dim=-1)
    return attention_weights @ values


In [79]:
torch.manual_seed(123)
attention = SelfAttention_v1(inputs.shape[-1],2)
attention.forward(inputs)

tensor([[0.3510, 0.8813],
        [0.3568, 0.8978],
        [0.3565, 0.8971],
        [0.3467, 0.8697],
        [0.3449, 0.8650],
        [0.3505, 0.8801]], grad_fn=<MmBackward0>)

In [84]:
import torch.nn as nn

class SelfAttention_v2(nn.Module):
  def __init__(self,d_in,d_out,bias = False):
    super().__init__()
    self.W_Q = nn.Linear(d_in,d_out,bias=bias)
    self.W_V = nn.Linear(d_in,d_out,bias=bias)
    self.W_K = nn.Linear(d_in,d_out,bias=bias)
  def forward(self,x):
    keys = self.W_K(x)
    values = self.W_V(x)
    queries = self.W_Q(x)
    attention_scores = queries @ keys.T
    attention_weights = torch.softmax(attention_scores/keys.shape[-1]**0.5,dim=-1)
    return attention_weights @ values


In [85]:
attention = SelfAttention_v2(inputs.shape[-1],2)
attention.forward(inputs)

tensor([[0.2673, 0.0036],
        [0.2668, 0.0036],
        [0.2668, 0.0036],
        [0.2694, 0.0035],
        [0.2691, 0.0036],
        [0.2688, 0.0035]], grad_fn=<MmBackward0>)